In [1]:
!pip install transformers qdrant-client torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 91.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.60.0
    Uninstalling grpcio-1.60.0:
      Successfully uninstalled grpcio-1.60.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requi

In [2]:
!pip install FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.3 MB/s eta 0:00:00
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.2.10-py3-none-any.whl size=166101 sha256=a7ca967cd8920ad2ec2e307b847d1bbce48d202d2d29ac273db635ab80bc83ba
  Stored in directory: /root/.cache/pip/wheels/3b/1d/d2/eec38cd59144f4c9767d7c55cfae8e8feec699071aa41ca5da
Successfully built FlagEmbedding


## Task3 : Implement Dense Retrival encoder using BGE and store dcuments embeddings into one of the vector DB mentioned above (Qdrant this sprint)

In [4]:
import logging
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct
from sentence_transformers import SentenceTransformer
import numpy as np
import os

from qdrant_client import QdrantClient
from qdrant_client.http import models as rest
from FlagEmbedding import FlagModel
from qdrant_client.http.models import VectorParams, Distance
from FlagEmbedding import FlagReranker

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-24 21:54:29.022917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 21:54:29.023082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 21:54:29.168361: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# Set up logging
logging.basicConfig(level=logging.INFO)

In [8]:
def get_qdrant_client_object(api_key, qdrant_url):
    """
    Create and return a QdrantClient object for interacting with a Qdrant database.

    Args:
    - api_key (str): The API key for authentication with the Qdrant database.
    - qdrant_url (str): The URL of the Qdrant database.

    Returns:
    - QdrantClient: An instance of QdrantClient configured to interact with the specified Qdrant database.
    """
    qdrant_client = QdrantClient(
        url=qdrant_url,
        api_key=api_key
    )
    return qdrant_client


In [9]:
def create_qdrant_collection_if_not_exist(qdrant_client, collection_name):
    """
    Create a Qdrant collection if it does not already exist.

    Args:
    - qdrant_client (QdrantClient): The QdrantClient object for interacting with the Qdrant database.
    - collection_name (str): The name of the collection to create or check for existence.

    Returns:
    - QdrantClient: The updated QdrantClient object.
    """
    try:
        if not qdrant_client.collection_exists(collection_name):
            qdrant_client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
            )
            logging.info(f"Collection '{collection_name}' created.")
        else:
            logging.info(f"Collection '{collection_name}' already exists.")
    except Exception as e:
        logging.error(f"Failed to create collection '{collection_name}': {e}")

    return qdrant_client

    

In [10]:
def create_bge_model(model_type="BAAI/bge-reranker-v2-m3"):
    """
    Create a BGE (Bing General Evaluator) model for reranking.

    Args:
    - model_type (str): The model type to use. Default is "BAAI/bge-reranker-v2-m3".

    Returns:
    - FlagModel: A FlagModel instance for the specified BGE model.
    """
    bgeModel = FlagModel(model_type, use_fp16=True)
    return bgeModel


In [11]:
def create_embedding_for_document(document, embedding_model):
    """
    Create an embedding for a given document using an embedding model.

    Args:
    - document (str): The document text to create an embedding for.
    - embedding_model: The embedding model to use for encoding the document.

    Returns:
    - np.array: A numpy array representing the embedding of the document.
    """
    embedding = embedding_model.encode(document)
    return embedding


In [12]:
def add_embedding_to_qdrant(qdrant_client, collection_name, embedding_vector, document, doc_id):
    """
    Add an embedding vector and corresponding document information to a Qdrant collection.

    Args:
    - qdrant_client: The Qdrant client used to interact with the Qdrant database.
    - collection_name (str): The name of the collection in which to add the embedding.
    - embedding_vector (List[float]): The embedding vector to add to the collection.
    - document (str): The document associated with the embedding.
    - doc_id (str): The ID of the document.

    Returns:
    - None
    """
    """
    Add an embedding vector and corresponding document information to a Qdrant collection.
    """
    try:
        qdrant_client.upsert(
            collection_name=collection_name,
            points=[PointStruct(
                id=doc_id,
                vector=embedding_vector,
                payload={"text": document}
            )]
        )
        logging.info(f"Document ID {doc_id} added to collection '{collection_name}'.")
    except Exception as e:
        logging.error(f"Failed to add document ID {doc_id} to collection '{collection_name}': {e}")


In [13]:
def add_document_to_qdrant(qdrant_client, collection_name, embedding_model, documents, start_index):
    """
    Add a list of documents to a Qdrant collection along with their embeddings.

    Args:
    - qdrant_client: The Qdrant client used to interact with the Qdrant database.
    - collection_name (str): The name of the collection in which to add the documents.
    - embedding_model: The embedding model to use for encoding the documents.
    - documents (List[str]): A list of document texts to add to the collection.
    - start_index (int): The starting index for the document IDs.

    Returns:
    - None
    """
    for idx, document in enumerate(documents):
        embedding_vector = create_embedding_for_document(document, embedding_model)
        add_embedding_to_qdrant(qdrant_client, collection_name, embedding_vector, document, idx + start_index)


In [14]:
### function to test usage of above functions
def main():
    api_key="RVO2uufS-EDtnf1FOUeCebwWGWvFJkbA8JWE7Z2PN7gAQpbdVKDVWQ"
    qdrant_url ="https://c69792c8-ab50-4bfb-bf68-bd9ae6fb16f9.us-east4-0.gcp.cloud.qdrant.io"
    qdrant_client=get_qdrant_client_object(api_key,qdrant_url)
    bge_model=create_bge_model()
    documents = [
                    "Retrieval-Augmented Generation (RAG) combines retrieval and generation for NLP tasks.",
                    "Generative Pre-trained Transformer (GPT) is a language generation model.",
                    "RAG uses a retriever to fetch relevant documents and a generator to produce answers.",
                    "BERT is a model designed for understanding the context of a word in a sentence."
                ]
    qdrant_client=create_qdrant_collection_if_not_exist(qdrant_client,"first_collection")
    add_document_to_qdrant(qdrant_client,"first_collection",bge_model,documents,0)
    return qdrant_client
    

In [15]:
qdrant_client=main()

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at BAAI/bge-reranker-v2-m3 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## task 6 : Implement Reranker using BGE.

In [16]:
def create_BGE_reranker(model_type='BAAI/bge-reranker-v2-m3'):
    """
    Create a Bing General Evaluator (BGE) reranker.

    Args:
    - model_type (str): The model type to use. Default is 'BAAI/bge-reranker-v2-m3'.

    Returns:
    - FlagReranker: A FlagReranker instance for the specified BGE reranker model.
    """
    reranker = FlagReranker(model_type, use_fp16=True)
    return reranker


In [18]:
def search_similar_vectors(qdrant_client, collection_name, query_vector, limit=10):
    """
    Search for similar vectors to a query vector in a Qdrant collection.

    Args:
    - qdrant_client: The Qdrant client used to interact with the Qdrant database.
    - collection_name (str): The name of the collection to search in.
    - query_vector (List[float]): The query vector for which to find similar vectors.
    - limit (int): The maximum number of results to return. Default is 10.

    Returns:
    - dict: A dictionary containing the search results.
    """
    try:
        return qdrant_client.search(
            collection_name=collection_name, query_vector=query_vector, limit=limit
        )
    except Exception as e:
        logging.error(f"Failed to search similar vectors: {e}")
        return []


In [19]:
def retrieve_text_by_ids(qdrant_client, collection_name, point_ids):
    """
    Retrieve text associated with specific IDs from a Qdrant collection.

    Args:
    - qdrant_client: The Qdrant client used to interact with the Qdrant database.
    - collection_name (str): The name of the collection to retrieve from.
    - point_ids (List[str]): A list of IDs for which to retrieve text.

    Returns:
    - List[str]: A list of texts associated with the specified IDs.
    """
    try:
        results = qdrant_client.retrieve(collection_name=collection_name, ids=ids)
        return [result.payload["text"] for result in results]
    except Exception as e:
        logging.error(f"Failed to retrieve texts by IDs: {e}")
        return []


In [20]:
def rank_texts(reranker_object, query, texts):
    """
    Rank a list of texts based on their relevance to a query using a reranker object.

    Args:
    - reranker_object: The reranker object used to compute scores.
    - query (str): The query text for which to rank the texts.
    - texts (List[str]): A list of text documents to be ranked.

    Returns:
    - List[str]: A list of text documents sorted by their relevance to the query.
    """
    scores = reranker_object.compute_score([[query, text] for text in texts])
    text_score_pairs = list(zip(texts, scores))
    sorted_text_score_pairs = sorted(text_score_pairs, key=lambda x: x[1], reverse=True)
    sorted_texts = [pair[0] for pair in sorted_text_score_pairs]
    return sorted_texts


In [ ]:
#### Usage of above function 
bge_model=create_bge_model()
reranker_object=create_BGE_reranker()
query="What is RAG"
query_vector=create_embedding_for_document(query,bge_model)
similar_vectors = search_similar_vectors(qdrant_client, "first_collection", query_vector)
similar_ids=[result.id for result in similar_vectors ]
similar_texts=retrieve_text_by_ids(qdrant_client, "first_collection", similar_ids)
print(similar_texts)
similar_texts=rank_texts(reranker_object,query, similar_texts)
similar_texts